<a href="https://colab.research.google.com/github/nurimammasri/Dicoding-Belajar-Pengembangan-Machine-Learning/blob/main/17_TFDS_exporting_your_input_into_training_pipelines.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np


import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [6]:
import logging
logging.getLogger('tensorflow').disabled = True


In [7]:
# url = 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
url = 'https://github.com/nurimammasri/Dicoding-Belajar-Pengembangan-Machine-Learning/raw/main/data/diamonds.csv'
df = pd.read_csv(url)

In [8]:
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### Split Dataset menjadi train, test, dan validation

In [9]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [10]:
#Membuat target dari prediksi model
# price 0 = low; 1 = high
df['target'] = np.where(df['price']==327, 0, 1)

# Drop un-used columns.
df = df.drop(columns=['price'])

### Create input pipeline using tf data

In [11]:
# Cara untuk membuat dataset tf.data dari pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [12]:
batch_size = 10 #bath ukuran kecil untuk demonstrasi
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [13]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of carat:', feature_batch['carat'])
  print('A batch of targets:', label_batch )

Every feature: ['Unnamed: 0', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']
A batch of carat: tf.Tensor([0.38 1.22 0.31 0.73 0.87 0.9  0.46 0.3  0.3  0.42], shape=(10,), dtype=float64)
A batch of targets: tf.Tensor([1 1 1 1 1 1 1 1 1 1], shape=(10,), dtype=int64)


In [14]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())


In [15]:
#numeric column
carat = feature_column.numeric_column('carat')
demo(carat)

[[0.73]
 [0.27]
 [0.6 ]
 [0.7 ]
 [2.01]
 [1.2 ]
 [1.04]
 [1.01]
 [1.01]
 [1.43]]


In [16]:
#bucketized column 
carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1, 2])
demo(carat_buckets)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [17]:
#categorical
color_type = feature_column.categorical_column_with_vocabulary_list(
      'color', ['E', 'I','J','D','H', 'G','F'])

color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)


[[0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0.]]


In [18]:
#embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

[[ 0.17688434 -0.15628633 -0.04890993 -0.25763416  0.00214265 -0.30630141]
 [-0.36397153 -0.6798694   0.00111301  0.74484193  0.05711588  0.16786902]
 [ 0.17688434 -0.15628633 -0.04890993 -0.25763416  0.00214265 -0.30630141]
 [ 0.48159948 -0.2983528   0.19063549  0.18840143 -0.08657558  0.61336285]
 [ 0.17688434 -0.15628633 -0.04890993 -0.25763416  0.00214265 -0.30630141]
 [ 0.48159948 -0.2983528   0.19063549  0.18840143 -0.08657558  0.61336285]
 [ 0.48159948 -0.2983528   0.19063549  0.18840143 -0.08657558  0.61336285]
 [-0.5885631   0.10973608  0.17276712  0.03654357  0.4771163  -0.11643896]
 [-0.36397153 -0.6798694   0.00111301  0.74484193  0.05711588  0.16786902]
 [ 0.48159948 -0.2983528   0.19063549  0.18840143 -0.08657558  0.61336285]]


In [19]:
#hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket(
      'clarity', hash_bucket_size=5)
demo(feature_column.indicator_column(clarity_hashed))

[[0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [20]:
#cross feature
#data yang di cross harus berupa string, categorical, atau bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type],
                                                hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


In [21]:
#Pilih feature column mana yang akan digunakan
feature_columns = []

In [22]:
#embedding column
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=8)
feature_columns.append(clarity_embedding)

In [23]:
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))

In [24]:
#membuat feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [25]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [26]:
#create, compile, and train the model
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
1686/1686 [==============================] - 8s 4ms/step - loss: 8.7758e-04 - accuracy: 0.9996 - val_loss: 3.7346e-04 - val_accuracy: 1.0000
Epoch 2/10
1686/1686 [==============================] - 7s 4ms/step - loss: 4.5949e-04 - accuracy: 1.0000 - val_loss: 2.6118e-04 - val_accuracy: 1.0000
Epoch 3/10
1686/1686 [==============================] - 7s 4ms/step - loss: 2.3614e-04 - accuracy: 1.0000 - val_loss: 3.8523e-04 - val_accuracy: 1.0000
Epoch 4/10
1686/1686 [==============================] - 7s 4ms/step - loss: 3.2125e-04 - accuracy: 1.0000 - val_loss: 4.4121e-04 - val_accuracy: 1.0000
Epoch 5/10
1686/1686 [==============================] - 7s 4ms/step - loss: 2.8307e-04 - accuracy: 1.0000 - val_loss: 4.6570e-04 - val_accuracy: 1.0000
Epoch 6/10
1686/1686 [==============================] - 7s 4ms/step - loss: 5.4660e-04 - accuracy: 1.0000 - val_loss: 4.0285e-04 - val_accuracy: 1.0000
Epoch 7/10
1686/1686 [==============================] - 8s 4ms/step - loss: 4.6707e-04 -

In [27]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1686/1686 [==============================] - 3s 2ms/step - loss: 3.3508e-04 - accuracy: 1.0000
Accuracy 0.9999814629554749
